In [9]:
%%time

# ===========================================================
# Stacking OOF: XGB + LGBM + CatBoost + RF (+ LogReg) -> Meta LogReg
# Métrica objetivo: PR-AUC  | Guarda metrics y predicciones
# ===========================================================
import os, time, json, joblib
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import (
    accuracy_score, f1_score, roc_auc_score,
    average_precision_score, brier_score_loss, classification_report, confusion_matrix
)
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.calibration import CalibratedClassifierCV

from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

import warnings
warnings.filterwarnings("ignore", message="X does not have valid feature names", category=UserWarning)

DATA_PATH = "../datasets/final/ico_dataset_final_v2_clean_enriquecido_feature_engineering_preico_v1.csv"
EXP_DIR = f"experiments/stacking_v1"
os.makedirs(EXP_DIR, exist_ok=True)

# ---------- carga
df = pd.read_csv(DATA_PATH)
target = "ico_successful"
df[target] = df[target].astype(int)
df = df.dropna(subset=[target])
print(f"Dataset cargado → {df.shape[0]} filas, {df.shape[1]} columnas")

# ---------- columnas (adaptá si cambió algo)
cat_cols = [c for c in ["platform","category","location"] if c in df.columns]
bin_cols = [c for c in ["mvp","has_twitter","has_facebook","is_tax_regulated","has_github",
                        "has_reddit","has_website","has_whitepaper","kyc",
                        "accepts_BTC","accepts_ETH","has_contract_address"] if c in df.columns]
num_cols = [c for c in df.columns if c not in cat_cols + bin_cols + [target]]

# ---------- preprocess para arboles (sin scaler) y para logreg (con scaler)
pre_tree = ColumnTransformer(
    transformers=[
        ("num", Pipeline([("imp", SimpleImputer(strategy="median"))]), num_cols),
        ("cat", Pipeline([("imp", SimpleImputer(strategy="most_frequent")),
                          ("ohe", OneHotEncoder(handle_unknown="ignore", drop="first"))]), cat_cols),
        ("bin", Pipeline([("imp", SimpleImputer(strategy="constant", fill_value=0))]), bin_cols),
    ],
    remainder="drop", sparse_threshold=1.0
)

pre_log = ColumnTransformer(
    transformers=[
        ("num", Pipeline([("imp", SimpleImputer(strategy="median")), ("sc", StandardScaler())]), num_cols),
        ("cat", Pipeline([("imp", SimpleImputer(strategy="most_frequent")),
                          ("ohe", OneHotEncoder(handle_unknown="ignore", drop="first"))]), cat_cols),
        ("bin", Pipeline([("imp", SimpleImputer(strategy="constant", fill_value=0))]), bin_cols),
    ],
    remainder="drop", sparse_threshold=0.3
)

X = df.drop(columns=[target]); y = df[target]
X_tr, X_te, y_tr, y_te = train_test_split(X, y, test_size=0.30, stratify=y, random_state=42)
print(f"Filas train: {X_tr.shape[0]} | test: {X_te.shape[0]} | vars: {X_tr.shape[1]}")

# ---------- bases (parámetros razonables; ajustá según tus mejores)
base_models = {
    "xgb": Pipeline([("pre", pre_tree),
                     ("clf", XGBClassifier(
                         objective="binary:logistic", eval_metric="logloss",
                         tree_method="hist", random_state=42, n_jobs=1,
                         n_estimators=1000, learning_rate=0.03, max_depth=4,
                         min_child_weight=3, subsample=0.9, colsample_bytree=0.8, reg_lambda=2.0))]),
    "lgbm": Pipeline([("pre", pre_tree),
                      ("clf", LGBMClassifier(
                          objective="binary", random_state=42, n_jobs=1, verbose=-1,
                          n_estimators=1500, learning_rate=0.03, num_leaves=63,
                          min_child_samples=80, feature_fraction=0.8, bagging_fraction=0.8, bagging_freq=1))]),
    "cat": Pipeline([("pre", pre_tree),
                     ("clf", CatBoostClassifier(
                         loss_function="Logloss", eval_metric="PRAUC", random_state=42,
                         iterations=1500, learning_rate=0.03, depth=6, l2_leaf_reg=5.0,
                         verbose=False, allow_writing_files=False))]),
    "rf":  Pipeline([("pre", pre_tree),
                     ("clf", __import__("sklearn").ensemble.RandomForestClassifier(
                         n_estimators=1000, max_depth=None, min_samples_leaf=2, n_jobs=1, random_state=42))]),
    "log": Pipeline([("pre", pre_log),
                     ("clf", LogisticRegression(
                         C=1.0, solver="lbfgs", max_iter=2000, n_jobs=1))])
}

# ---------- OOF para meta
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
oof_meta = pd.DataFrame(index=X_tr.index)
test_meta = pd.DataFrame(index=X_te.index)

for name, pipe in base_models.items():
    oof_pred = np.zeros(X_tr.shape[0], dtype=float)
    test_pred = np.zeros(X_te.shape[0], dtype=float)
    for tr_idx, va_idx in skf.split(X_tr, y_tr):
        X_tr_i, X_va_i = X_tr.iloc[tr_idx], X_tr.iloc[va_idx]
        y_tr_i, y_va_i = y_tr.iloc[tr_idx], y_tr.iloc[va_idx]
        pipe.fit(X_tr_i, y_tr_i)
        oof_pred[va_idx] = pipe.predict_proba(X_va_i)[:, 1]
    # fit en todo el train y predecí test
    pipe.fit(X_tr, y_tr)
    test_pred[:] = pipe.predict_proba(X_te)[:, 1]
    oof_meta[name] = oof_pred
    test_meta[name] = test_pred
    # opcional: guardar cada base ya entrenada en full train
    joblib.dump(pipe, os.path.join(EXP_DIR, f"base_{name}.pkl"))

# ---------- meta-modelo (logistic) + calibración isotónica
meta = LogisticRegression(C=1.0, solver="lbfgs", max_iter=1000)
cal_meta = CalibratedClassifierCV(estimator=meta, method="isotonic", cv=5)
cal_meta.fit(oof_meta, y_tr)

y_proba = cal_meta.predict_proba(test_meta)[:, 1]
y_pred  = (y_proba >= 0.5).astype(int)

# ---------- métricas y guardado
m = {
    "model": "stacking_oof_hybrib",
    "roc_auc": float(roc_auc_score(y_te, y_proba)),
    "pr_auc": float(average_precision_score(y_te, y_proba)),
    "f1": float(f1_score(y_te, y_pred)),
    "accuracy": float(accuracy_score(y_te, y_pred)),
    "brier": float(brier_score_loss(y_te, y_proba)),
    "confusion_matrix": confusion_matrix(y_te, y_pred).tolist(),
}

print("\n=== STACKING (meta logreg calibrada) ===")
print(f"ROC-AUC: {m['roc_auc']:.4f} | PR-AUC: {m['pr_auc']:.4f} | F1: {m['f1']:.4f} | Acc: {m['accuracy']:.4f} | Brier: {m['brier']:.4f}")
print("\nClassification Report:\n", classification_report(y_te, y_pred, digits=4))
print("\nConfusion Matrix:\n", np.array(m["confusion_matrix"]))

if y_proba is not None:
    print("\nProb-metrics:")
    print(f"ROC-AUC : {roc_auc_score(y_te, y_proba):.4f}")
    print(f"PR-AUC  : {average_precision_score(y_te, y_proba):.4f}")
    print(f"Brier   : {brier_score_loss(y_te, y_proba):.4f}")
        
# guardar artefactos
with open(os.path.join(EXP_DIR, "metrics.json"), "w") as f:
    json.dump({"stacking_oof_hybrib": m}, f, indent=2)
pd.DataFrame({"y_true": y_te.values, "y_proba": y_proba, "y_pred": y_pred}).to_csv(
    os.path.join(EXP_DIR, "test_predictions.csv"), index=False
)
joblib.dump(cal_meta, os.path.join(EXP_DIR, "stacking_meta.pkl"))
print("\nArtefactos en:", EXP_DIR)


Dataset cargado → 4457 filas, 30 columnas
Filas train: 3119 | test: 1338 | vars: 29

=== STACKING (meta logreg calibrada) ===
ROC-AUC: 0.8842 | PR-AUC: 0.8681 | F1: 0.7635 | Acc: 0.8139 | Brier: 0.1342

Classification Report:
               precision    recall  f1-score   support

           0     0.7924    0.9087    0.8466       756
           1     0.8535    0.6907    0.7635       582

    accuracy                         0.8139      1338
   macro avg     0.8229    0.7997    0.8051      1338
weighted avg     0.8190    0.8139    0.8105      1338


Confusion Matrix:
 [[687  69]
 [180 402]]

Prob-metrics:
ROC-AUC : 0.8842
PR-AUC  : 0.8681
Brier   : 0.1342

Artefactos en: experiments/stacking_v1
CPU times: total: 5min 54s
Wall time: 1min 38s


In [13]:
# ===========================================
# Stacking OOF con meta-features + meta-XGB + blending
# (versión coherente, sin errores de índice)
# ===========================================
import warnings, os, time, json, joblib
import numpy as np
import pandas as pd

from sklearn.model_selection import StratifiedKFold, train_test_split, RandomizedSearchCV
from sklearn.metrics import (average_precision_score, roc_auc_score, f1_score,
                             accuracy_score, brier_score_loss, classification_report, confusion_matrix)
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from scipy.special import logit
from scipy.optimize import minimize
from sklearn.preprocessing import StandardScaler

# --- silenciar warning de feature names de LGBM
warnings.filterwarnings("ignore", message="X does not have valid feature names", category=UserWarning)

# ---------- carga
DATA_PATH = "../datasets/final/ico_dataset_final_v2_clean_enriquecido_feature_engineering_preico_v1.csv"
df = pd.read_csv(DATA_PATH)
target = "ico_successful"
df[target] = df[target].astype(int)
df = df.dropna(subset=[target])

# ---------- columnas (ajustá si cambió algo)
cat_cols = [c for c in ["platform","category","location","caps_unit"] if c in df.columns]
bin_cols = [c for c in ["mvp","has_twitter","has_facebook","is_tax_regulated","has_github",
                        "has_reddit","has_website","has_whitepaper","kyc",
                        "accepts_BTC","accepts_ETH","has_contract_address"] if c in df.columns]
num_cols = [c for c in df.columns if c not in cat_cols + bin_cols + [target]]

# ---------- preprocess para árboles (sin scaler) y para logreg (con scaler)
pre_tree = ColumnTransformer(
    transformers=[
        ("num", Pipeline([("imp", SimpleImputer(strategy="median"))]), num_cols),
        ("cat", Pipeline([("imp", SimpleImputer(strategy="most_frequent")),
                          ("ohe", OneHotEncoder(handle_unknown="ignore", drop="first"))]), cat_cols),
        ("bin", Pipeline([("imp", SimpleImputer(strategy="constant", fill_value=0))]), bin_cols),
    ],
    remainder="drop", sparse_threshold=1.0
)
pre_log = ColumnTransformer(
    transformers=[
        ("num", Pipeline([("imp", SimpleImputer(strategy="median")), ("sc", StandardScaler())]), num_cols),
        ("cat", Pipeline([("imp", SimpleImputer(strategy="most_frequent")),
                          ("ohe", OneHotEncoder(handle_unknown="ignore", drop="first"))]), cat_cols),
        ("bin", Pipeline([("imp", SimpleImputer(strategy="constant", fill_value=0))]), bin_cols),
    ],
    remainder="drop", sparse_threshold=0.3
)

# ---------- split base
X = df.drop(columns=[target]); y = df[target]
X_tr, X_te, y_tr, y_te = train_test_split(X, y, test_size=0.30, stratify=y, random_state=42)
print(f"Train: {X_tr.shape}, Test: {X_te.shape}")

# ---------- modelos base (parámetros razonables)
base_models = {
    "xgb": Pipeline([("pre", pre_tree),
                     ("clf", XGBClassifier(
                         objective="binary:logistic", eval_metric="logloss",
                         tree_method="hist", random_state=42, n_jobs=1,
                         n_estimators=800, learning_rate=0.03, max_depth=4,
                         min_child_weight=3, subsample=0.9, colsample_bytree=0.8, reg_lambda=2.0))]),
    "lgbm": Pipeline([("pre", pre_tree),
                      ("clf", LGBMClassifier(
                          objective="binary", random_state=42, n_jobs=1, verbose=-1,
                          n_estimators=1200, learning_rate=0.03, num_leaves=63,
                          min_child_samples=80, feature_fraction=0.8, bagging_fraction=0.8, bagging_freq=1))]),
    "cat": Pipeline([("pre", pre_tree),
                     ("clf", CatBoostClassifier(
                         loss_function="Logloss",
                         eval_metric="PRAUC",      # <— compatible con versiones más viejas
                         iterations=1200, learning_rate=0.03, depth=6, l2_leaf_reg=5.0,
                         random_state=42, verbose=False, allow_writing_files=False))]),
    "rf":  Pipeline([("pre", pre_tree),
                     ("clf", RandomForestClassifier(
                         n_estimators=600, max_depth=None, min_samples_leaf=2, n_jobs=1, random_state=42))]),
    "log": Pipeline([("pre", pre_log),
                     ("clf", LogisticRegression(
                         C=1.0, solver="lbfgs", max_iter=2000, n_jobs=1))]),
}

# ---------- OOF sin perder alineación
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
oof_meta = pd.DataFrame(index=X_tr.index)
test_meta = pd.DataFrame(index=X_te.index)

for name, pipe in base_models.items():
    oof_pred = np.zeros(X_tr.shape[0], dtype=float)
    test_pred = np.zeros(X_te.shape[0], dtype=float)
    for tr_idx, va_idx in skf.split(X_tr, y_tr):
        # usar posiciones -> luego mapear a índices
        X_tr_i, X_va_i = X_tr.iloc[tr_idx], X_tr.iloc[va_idx]
        y_tr_i, y_va_i = y_tr.iloc[tr_idx], y_tr.iloc[va_idx]
        pipe.fit(X_tr_i, y_tr_i)
        oof_pred[va_idx] = pipe.predict_proba(X_va_i)[:, 1]
    # fit final en todo el train y predecir test
    pipe.fit(X_tr, y_tr)
    test_pred[:] = pipe.predict_proba(X_te)[:, 1]
    oof_meta[name] = oof_pred
    test_meta[name] = test_pred

# ---------- meta-features (proba + logit + rank)
def safe_logit(p, eps=1e-6):
    p = np.clip(p, eps, 1 - eps)
    return logit(p)

def rank01(a):
    order = pd.Series(a).rank(method="average")
    return (order - order.min()) / (order.max() - order.min() + 1e-12)

use_cols = [c for c in ["xgb","lgbm","cat","rf","log"] if c in oof_meta.columns]
Z_tr = oof_meta[use_cols].copy()
Z_te = test_meta[use_cols].copy()

for c in use_cols:
    Z_tr[f"{c}_logit"] = safe_logit(Z_tr[c].values)
    Z_tr[f"{c}_rank"]  = rank01(Z_tr[c].values)
    Z_te[f"{c}_logit"] = safe_logit(Z_te[c].values)
    Z_te[f"{c}_rank"]  = rank01(Z_te[c].values)

logit_rank_cols = [c for c in Z_tr.columns if c.endswith("_logit") or c.endswith("_rank")]
sc = StandardScaler()
Z_tr[logit_rank_cols] = sc.fit_transform(Z_tr[logit_rank_cols])
Z_te[logit_rank_cols] = sc.transform(Z_te[logit_rank_cols])

print("Meta features:", list(Z_tr.columns))

# ---------- meta-XGB con early stopping (uso split por POSICIONES, no por índices)
idx_all = np.arange(len(Z_tr))
idx_tr_in, idx_val_in = train_test_split(idx_all, test_size=0.2, stratify=y_tr.values, random_state=42)

Z_tr_in, Z_val_in = Z_tr.iloc[idx_tr_in], Z_tr.iloc[idx_val_in]
y_tr_in, y_val_in = y_tr.iloc[idx_tr_in], y_tr.iloc[idx_val_in]

meta_xgb = XGBClassifier(
    objective="binary:logistic",
    eval_metric="aucpr",
    tree_method="hist",
    n_jobs=1,
    random_state=42,
)

param_dist = {
    "n_estimators": [300, 500, 800, 1000],
    "learning_rate": [0.02, 0.03, 0.05, 0.1],
    "max_depth": [2, 3],
    "min_child_weight": [1, 3, 5],
    "subsample": [0.7, 0.85, 1.0],
    "colsample_bytree": [0.6, 0.8, 1.0],
    "reg_lambda": [0.5, 1.0, 2.0],
}

rs = RandomizedSearchCV(
    meta_xgb, param_distributions=param_dist, n_iter=25,
    scoring="average_precision", cv=3, random_state=42, n_jobs=4, refit=False, verbose=0
)
rs.fit(Z_tr_in, y_tr_in)
best_params = rs.best_params_

meta_best = XGBClassifier(objective="binary:logistic", eval_metric="aucpr", tree_method="hist",
                          n_jobs=1, random_state=42, **best_params)
meta_best.fit(
    Z_tr_in, y_tr_in,
    eval_set=[(Z_val_in, y_val_in)],
    verbose=False,
    #early_stopping_rounds=100
)
proba_meta = meta_best.predict_proba(Z_te)[:, 1]

# ---------- blending (pesos óptimos) — usar POSICIONES
P_tr = oof_meta[use_cols].values   # (n_tr, K)
P_te = test_meta[use_cols].values  # (n_te, K)
y_tr_arr = y_tr.values
y_te_arr = y_te.values

P_tr_in = P_tr[idx_tr_in, :]
P_val_in = P_tr[idx_val_in, :]
y_tr_in_arr = y_tr_arr[idx_tr_in]
y_val_in_arr = y_tr_arr[idx_val_in]

def ap_neg(w):
    w = np.clip(w, 0, None)
    s = w.sum()
    if s <= 1e-12:
        return 1.0
    w = w / s
    p = P_val_in @ w
    return -average_precision_score(y_val_in_arr, p)

K = P_tr.shape[1]
w0 = np.ones(K) / K
bounds = [(0.0, 1.0)] * K
cons = ({"type":"eq", "fun": lambda w: np.sum(np.clip(w,0,None)) - 1.0},)

opt = minimize(ap_neg, w0, method="SLSQP", bounds=bounds, constraints=cons, options={"maxiter":200, "ftol":1e-8})
w_star = np.clip(opt.x, 0, None); w_star /= (w_star.sum() + 1e-12)

proba_blend = P_te @ w_star

# ---------- híbrido (promedio ponderado meta vs blend)
alpha = 0.5
proba_hybrid = alpha * proba_meta + (1 - alpha) * proba_blend

def metrics_from_proba(y_true, y_proba, thr=0.5):
    y_pred = (y_proba >= thr).astype(int)
    return dict(
        pr_auc=float(average_precision_score(y_true, y_proba)),
        roc_auc=float(roc_auc_score(y_true, y_proba)),
        f1=float(f1_score(y_true, y_pred)),
        accuracy=float(accuracy_score(y_true, y_pred)),
        brier=float(brier_score_loss(y_true, y_proba)),
        confusion_matrix=confusion_matrix(y_true, y_pred).tolist(),
    )

m_meta   = metrics_from_proba(y_te_arr, proba_meta)
m_blend  = metrics_from_proba(y_te_arr, proba_blend)
m_hybrid = metrics_from_proba(y_te_arr, proba_hybrid)

print("\n[Meta-XGB]    ", m_meta)
print("[Blend]       ", m_blend)
print(f"[Hybrid a={alpha}] ", m_hybrid)
print("\nPesos blend:")
for name, w in zip(use_cols, w_star):
    print(f"  {name:>8s}: {w:.3f}")

# ---------- elegir mejor por PR-AUC
best_name, best_proba, best_m = max(
    [("meta_xgb", proba_meta, m_meta), ("blend", proba_blend, m_blend), (f"hybrid_{alpha}", proba_hybrid, m_hybrid)],
    key=lambda t: t[2]["pr_auc"]
)

print(f"\n>>> Mejor meta: {best_name} | PR-AUC={best_m['pr_auc']:.4f} ROC-AUC={best_m['roc_auc']:.4f} Brier={best_m['brier']:.4f}")
print("\nClassification Report:\n", classification_report(y_te_arr, (best_proba>=0.5).astype(int), digits=4))


Train: (3119, 29), Test: (1338, 29)
Meta features: ['xgb', 'lgbm', 'cat', 'rf', 'log', 'xgb_logit', 'xgb_rank', 'lgbm_logit', 'lgbm_rank', 'cat_logit', 'cat_rank', 'rf_logit', 'rf_rank', 'log_logit', 'log_rank']

[Meta-XGB]     {'pr_auc': 0.8664949034833431, 'roc_auc': 0.8825205912834779, 'f1': 0.7674858223062382, 'accuracy': 0.8161434977578476, 'brier': 0.13543348333999988, 'confusion_matrix': [[686, 70], [176, 406]]}
[Blend]        {'pr_auc': 0.8625960655287102, 'roc_auc': 0.8800910016545755, 'f1': 0.764378478664193, 'accuracy': 0.8101644245142003, 'brier': 0.13887237819930862, 'confusion_matrix': [[672, 84], [170, 412]]}
[Hybrid a=0.5]  {'pr_auc': 0.8670382623100331, 'roc_auc': 0.8839137984326988, 'f1': 0.7679245283018868, 'accuracy': 0.8161434977578476, 'brier': 0.13565365214192857, 'confusion_matrix': [[685, 71], [175, 407]]}

Pesos blend:
       xgb: 0.200
      lgbm: 0.200
       cat: 0.200
        rf: 0.200
       log: 0.200

>>> Mejor meta: hybrid_0.5 | PR-AUC=0.8670 ROC-AUC=0

In [15]:
# ===========================================
# Stacking OOF con meta-features + meta-XGB + blending
# (versión coherente, sin errores de índice)
# ===========================================
import warnings, os, time, json, joblib
import numpy as np
import pandas as pd

from sklearn.model_selection import StratifiedKFold, RepeatedStratifiedKFold, train_test_split, RandomizedSearchCV
from sklearn.metrics import (average_precision_score, roc_auc_score, f1_score,
                             accuracy_score, brier_score_loss, classification_report, confusion_matrix)
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from scipy.special import logit
from scipy.optimize import minimize
from sklearn.preprocessing import StandardScaler

# --- silenciar warning de feature names de LGBM
warnings.filterwarnings("ignore", message="X does not have valid feature names", category=UserWarning)

# ---------- carga
DATA_PATH = "../datasets/final/ico_dataset_final_v2_clean_enriquecido_feature_engineering_preico_v1.csv"
df = pd.read_csv(DATA_PATH)
target = "ico_successful"
df[target] = df[target].astype(int)
df = df.dropna(subset=[target])

# ---------- columnas (ajustá si cambió algo)
cat_cols = [c for c in ["platform","category","location","caps_unit"] if c in df.columns]
bin_cols = [c for c in ["mvp","has_twitter","has_facebook","is_tax_regulated","has_github",
                        "has_reddit","has_website","has_whitepaper","kyc",
                        "accepts_BTC","accepts_ETH","has_contract_address"] if c in df.columns]
num_cols = [c for c in df.columns if c not in cat_cols + bin_cols + [target]]

# ---------- preprocess para árboles (sin scaler) y para logreg (con scaler)
pre_tree = ColumnTransformer(
    transformers=[
        ("num", Pipeline([("imp", SimpleImputer(strategy="median"))]), num_cols),
        ("cat", Pipeline([("imp", SimpleImputer(strategy="most_frequent")),
                          ("ohe", OneHotEncoder(handle_unknown="ignore", drop="first"))]), cat_cols),
        ("bin", Pipeline([("imp", SimpleImputer(strategy="constant", fill_value=0))]), bin_cols),
    ],
    remainder="drop", sparse_threshold=1.0
)
pre_log = ColumnTransformer(
    transformers=[
        ("num", Pipeline([("imp", SimpleImputer(strategy="median")), ("sc", StandardScaler())]), num_cols),
        ("cat", Pipeline([("imp", SimpleImputer(strategy="most_frequent")),
                          ("ohe", OneHotEncoder(handle_unknown="ignore", drop="first"))]), cat_cols),
        ("bin", Pipeline([("imp", SimpleImputer(strategy="constant", fill_value=0))]), bin_cols),
    ],
    remainder="drop", sparse_threshold=0.3
)

# ---------- split base
X = df.drop(columns=[target]); y = df[target]
X_tr, X_te, y_tr, y_te = train_test_split(X, y, test_size=0.30, stratify=y, random_state=42)
print(f"Train: {X_tr.shape}, Test: {X_te.shape}")

# ---------- modelos base (parámetros razonables)
base_models = {
    "xgb": Pipeline([("pre", pre_tree),
                     ("clf", XGBClassifier(
                         objective="binary:logistic", eval_metric="logloss",
                         tree_method="hist", random_state=42, n_jobs=1,
                         n_estimators=1200, learning_rate=0.03, max_depth=4,
                         min_child_weight=3, subsample=0.9, colsample_bytree=0.8, reg_lambda=2.0))]),
    "lgbm": Pipeline([("pre", pre_tree),
                      ("clf", LGBMClassifier(
                          objective="binary", random_state=42, n_jobs=1, verbose=-1,
                          n_estimators=1200, learning_rate=0.03, num_leaves=63,
                          min_child_samples=80, feature_fraction=0.8, bagging_fraction=0.8, bagging_freq=1))]),
    "cat": Pipeline([("pre", pre_tree),
                     ("clf", CatBoostClassifier(
                         loss_function="Logloss",
                         eval_metric="PRAUC",      # <— compatible con versiones más viejas
                         iterations=1200, learning_rate=0.03, depth=6, l2_leaf_reg=5.0,
                         random_state=42, verbose=False, allow_writing_files=False))]),
    "rf":  Pipeline([("pre", pre_tree),
                     ("clf", RandomForestClassifier(
                         n_estimators=1200, max_depth=None, min_samples_leaf=2, n_jobs=1, random_state=42))]),
    "log": Pipeline([("pre", pre_log),
                     ("clf", LogisticRegression(
                         C=1.0, solver="lbfgs", max_iter=2000, n_jobs=1))]),
}

# ---------- OOF sin perder alineación
rskf = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=42)

oof_meta = pd.DataFrame(index=X_tr.index)
test_meta = pd.DataFrame(index=X_te.index)

for name, pipe in base_models.items():
    oof_sum = np.zeros(X_tr.shape[0], dtype=float)
    oof_cnt = np.zeros(X_tr.shape[0], dtype=float)
    test_sum = np.zeros(X_te.shape[0], dtype=float)

    for tr_idx, va_idx in rskf.split(X_tr, y_tr):
        Xtr, Xva = X_tr.iloc[tr_idx], X_tr.iloc[va_idx]
        ytr, yva = y_tr.iloc[tr_idx], y_tr.iloc[va_idx]
        pipe.fit(Xtr, ytr)
        p = pipe.predict_proba(Xva)[:,1]
        oof_sum[va_idx] += p
        oof_cnt[va_idx] += 1.0
        test_sum += pipe.predict_proba(X_te)[:,1] / rskf.get_n_splits()

    oof_meta[name] = np.divide(oof_sum, oof_cnt, out=np.zeros_like(oof_sum), where=oof_cnt>0)
    test_meta[name] = test_sum

# ---------- meta-features (proba + logit + rank)
def safe_logit(p, eps=1e-6):
    p = np.clip(p, eps, 1 - eps)
    return logit(p)

def rank01(a):
    order = pd.Series(a).rank(method="average")
    return (order - order.min()) / (order.max() - order.min() + 1e-12)

use_cols = [c for c in ["xgb","lgbm","cat","rf","log"] if c in oof_meta.columns]
Z_tr = oof_meta[use_cols].copy()
Z_te = test_meta[use_cols].copy()

for c in use_cols:
    Z_tr[f"{c}_logit"] = safe_logit(Z_tr[c].values)
    Z_tr[f"{c}_rank"]  = rank01(Z_tr[c].values)
    Z_te[f"{c}_logit"] = safe_logit(Z_te[c].values)
    Z_te[f"{c}_rank"]  = rank01(Z_te[c].values)

logit_rank_cols = [c for c in Z_tr.columns if c.endswith("_logit") or c.endswith("_rank")]
sc = StandardScaler()
Z_tr[logit_rank_cols] = sc.fit_transform(Z_tr[logit_rank_cols])
Z_te[logit_rank_cols] = sc.transform(Z_te[logit_rank_cols])

print("Meta features:", list(Z_tr.columns))

# ---------- meta-XGB con early stopping (uso split por POSICIONES, no por índices)
idx_all = np.arange(len(Z_tr))
idx_tr_in, idx_val_in = train_test_split(idx_all, test_size=0.2, stratify=y_tr.values, random_state=42)

Z_tr_in, Z_val_in = Z_tr.iloc[idx_tr_in], Z_tr.iloc[idx_val_in]
y_tr_in, y_val_in = y_tr.iloc[idx_tr_in], y_tr.iloc[idx_val_in]

meta_xgb = XGBClassifier(
    objective="binary:logistic",
    eval_metric="aucpr",
    tree_method="hist",
    n_jobs=1,
    random_state=42,
)

param_dist = {
    "n_estimators": [300, 500, 800, 1000],
    "learning_rate": [0.02, 0.03, 0.05, 0.1],
    "max_depth": [2, 3],
    "min_child_weight": [1, 3, 5],
    "subsample": [0.7, 0.85, 1.0],
    "colsample_bytree": [0.6, 0.8, 1.0],
    "reg_lambda": [0.5, 1.0, 2.0],
}

rs = RandomizedSearchCV(
    meta_xgb, param_distributions=param_dist, n_iter=25,
    scoring="average_precision", cv=3, random_state=42, n_jobs=4, refit=False, verbose=0
)
rs.fit(Z_tr_in, y_tr_in)
best_params = rs.best_params_

meta_best = XGBClassifier(objective="binary:logistic", eval_metric="aucpr", tree_method="hist",
                          n_jobs=1, random_state=42, **best_params)
meta_best.fit(
    Z_tr_in, y_tr_in,
    eval_set=[(Z_val_in, y_val_in)],
    verbose=False,
    #early_stopping_rounds=100
)
proba_meta = meta_best.predict_proba(Z_te)[:, 1]

# ---------- blending (pesos óptimos) — usar POSICIONES
P_tr = oof_meta[use_cols].values   # (n_tr, K)
P_te = test_meta[use_cols].values  # (n_te, K)
y_tr_arr = y_tr.values
y_te_arr = y_te.values

P_tr_in = P_tr[idx_tr_in, :]
P_val_in = P_tr[idx_val_in, :]
y_tr_in_arr = y_tr_arr[idx_tr_in]
y_val_in_arr = y_tr_arr[idx_val_in]

def ap_neg(w):
    w = np.clip(w, 0, None)
    s = w.sum()
    if s <= 1e-12:
        return 1.0
    w = w / s
    p = P_val_in @ w
    return -average_precision_score(y_val_in_arr, p)

K = P_tr.shape[1]
w0 = np.ones(K) / K
bounds = [(0.0, 1.0)] * K
cons = ({"type":"eq", "fun": lambda w: np.sum(np.clip(w,0,None)) - 1.0},)

opt = minimize(ap_neg, w0, method="SLSQP", bounds=bounds, constraints=cons, options={"maxiter":200, "ftol":1e-8})
w_star = np.clip(opt.x, 0, None); w_star /= (w_star.sum() + 1e-12)

proba_blend = P_te @ w_star

# ---------- híbrido (promedio ponderado meta vs blend)
alpha = 0.5
proba_hybrid = alpha * proba_meta + (1 - alpha) * proba_blend

def metrics_from_proba(y_true, y_proba, thr=0.5):
    y_pred = (y_proba >= thr).astype(int)
    return dict(
        pr_auc=float(average_precision_score(y_true, y_proba)),
        roc_auc=float(roc_auc_score(y_true, y_proba)),
        f1=float(f1_score(y_true, y_pred)),
        accuracy=float(accuracy_score(y_true, y_pred)),
        brier=float(brier_score_loss(y_true, y_proba)),
        confusion_matrix=confusion_matrix(y_true, y_pred).tolist(),
    )

m_meta   = metrics_from_proba(y_te_arr, proba_meta)
m_blend  = metrics_from_proba(y_te_arr, proba_blend)
m_hybrid = metrics_from_proba(y_te_arr, proba_hybrid)

print("\n[Meta-XGB]    ", m_meta)
print("[Blend]       ", m_blend)
print(f"[Hybrid a={alpha}] ", m_hybrid)
print("\nPesos blend:")
for name, w in zip(use_cols, w_star):
    print(f"  {name:>8s}: {w:.3f}")

# ---------- elegir mejor por PR-AUC
best_name, best_proba, best_m = max(
    [("meta_xgb", proba_meta, m_meta), ("blend", proba_blend, m_blend), (f"hybrid_{alpha}", proba_hybrid, m_hybrid)],
    key=lambda t: t[2]["pr_auc"]
)

print(f"\n>>> Mejor meta: {best_name} | PR-AUC={best_m['pr_auc']:.4f} ROC-AUC={best_m['roc_auc']:.4f} Brier={best_m['brier']:.4f}")
print("\nClassification Report:\n", classification_report(y_te_arr, (best_proba>=0.5).astype(int), digits=4))


Train: (3119, 29), Test: (1338, 29)
Meta features: ['xgb', 'lgbm', 'cat', 'rf', 'log', 'xgb_logit', 'xgb_rank', 'lgbm_logit', 'lgbm_rank', 'cat_logit', 'cat_rank', 'rf_logit', 'rf_rank', 'log_logit', 'log_rank']

[Meta-XGB]     {'pr_auc': 0.867691574089774, 'roc_auc': 0.8830035546100838, 'f1': 0.7700934579439253, 'accuracy': 0.8161434977578476, 'brier': 0.1348992476458625, 'confusion_matrix': [[680, 76], [170, 412]]}
[Blend]        {'pr_auc': 0.8630318013533859, 'roc_auc': 0.879834178803251, 'f1': 0.7663551401869159, 'accuracy': 0.8131539611360239, 'brier': 0.13918552965260406, 'confusion_matrix': [[678, 78], [172, 410]]}
[Hybrid a=0.5]  {'pr_auc': 0.8682478726777915, 'roc_auc': 0.8844774450444554, 'f1': 0.7642124883504194, 'accuracy': 0.8109118086696562, 'brier': 0.13539744398406436, 'confusion_matrix': [[675, 81], [172, 410]]}

Pesos blend:
       xgb: 0.200
      lgbm: 0.200
       cat: 0.200
        rf: 0.200
       log: 0.200

>>> Mejor meta: hybrid_0.5 | PR-AUC=0.8682 ROC-AUC=0.8

In [ ]:
ROC-AUC: 0.8842 | PR-AUC: 0.8681 | F1: 0.7635 | Acc: 0.8139 | Brier: 0.1342
{'pr_auc': 0.8670382623100331, 'roc_auc': 0.8839137984326988, 'f1': 0.7679245283018868, 'accuracy': 0.8161434977578476, 'brier': 0.13565365214192857, 'confusion_matrix': [[685, 71], [175, 407]]}
{'pr_auc': 0.8682478726777915, 'roc_auc': 0.8844774450444554, 'f1': 0.7642124883504194, 'accuracy': 0.8109118086696562, 'brier': 0.13539744398406436, 'confusion_matrix': [[675, 81], [172, 410]]}
{'pr_auc': 0.8720648161614923, 'roc_auc': 0.8854706449208166, 'f1': 0.7628865979381443, 'accuracy': 0.8109118086696562, 'brier': 0.13374609320060443, 'confusion_matrix': [[678, 78], [175, 407]]}

In [20]:
# ===========================================
# Boost al stacking: Blending + Meta Elastic-Net + Híbrido
# Requiere definidos: oof_meta (train OOF proba), test_meta (test proba),
#                     y_tr (Series), y_te (Series)
# ===========================================
import numpy as np, pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import (average_precision_score, roc_auc_score, f1_score,
                             accuracy_score, brier_score_loss, confusion_matrix)
from scipy.optimize import minimize
from scipy.special import logit
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegressionCV

def metrics_from_proba(y_true, y_proba, thr=0.5):
    y_pred = (y_proba >= thr).astype(int)
    return dict(
        pr_auc=float(average_precision_score(y_true, y_proba)),
        roc_auc=float(roc_auc_score(y_true, y_proba)),
        f1=float(f1_score(y_true, y_pred)),
        accuracy=float(accuracy_score(y_true, y_pred)),
        brier=float(brier_score_loss(y_true, y_proba)),
        confusion_matrix=confusion_matrix(y_true, y_pred).tolist(),
    )

# -------------- columnas disponibles (ajustá si cambian los nombres)
use_cols = [c for c in ["xgb","lgbm","cat","rf","log"] if c in oof_meta.columns]
assert len(use_cols) >= 2, "Necesito al menos 2 bases en oof_meta/test_meta."

P_tr = oof_meta[use_cols].values   # (n_train_meta, K)
P_te = test_meta[use_cols].values  # (n_test_meta , K)
y_tr_arr = y_tr.values
y_te_arr = y_te.values

# =============== (1) BLENDING: pesos óptimos por PR-AUC en holdout meta
# armamos un holdout pequeño del meta para optimizar pesos
idx_all = np.arange(P_tr.shape[0])
idx_tr_in, idx_val_in = train_test_split(idx_all, test_size=0.2, stratify=y_tr_arr, random_state=42)
P_tr_in, P_val_in = P_tr[idx_tr_in], P_tr[idx_val_in]
y_tr_in_arr, y_val_in_arr = y_tr_arr[idx_tr_in], y_tr_arr[idx_val_in]

def ap_neg_reg(w, lam=1e-3):
    # w >= 0, sum(w) = 1 (se normaliza dentro)
    w = np.clip(w, 0, None)
    s = w.sum()
    if s <= 1e-12: 
        return 1.0
    w = w / s
    p = P_val_in @ w
    # regularización L2 suave para evitar pesos extremos
    return -average_precision_score(y_val_in_arr, p) + lam * float((w**2).sum())

K = P_tr.shape[1]
bounds = [(0.0, 1.0)] * K
cons = ({"type":"eq", "fun": lambda w: np.sum(np.clip(w,0,None)) - 1.0},)

best_ap, best_w = -1.0, None
for seed in range(10):  # multi-start
    rng = np.random.default_rng(100 + seed)
    w0 = rng.random(K); w0 /= w0.sum()
    opt = minimize(ap_neg_reg, w0, method="SLSQP", bounds=bounds, constraints=cons,
                   options={"maxiter":300, "ftol":1e-9})
    w = np.clip(opt.x,0,None); w/= (w.sum()+1e-12)
    ap = -ap_neg_reg(w, lam=0.0)   # AP “real” sin penalización
    if ap > best_ap:
        best_ap, best_w = ap, w

w_star = best_w
proba_blend = P_te @ w_star
m_blend = metrics_from_proba(y_te_arr, proba_blend)

print("=== BLEND ===")
for name, w in zip(use_cols, w_star):
    print(f"  peso {name:>6s}: {w:.3f}")
print("metrics:", m_blend)

# =============== (2) META ELASTIC-NET sobre meta-features (proba + logit + rank)
def safe_logit(p, eps=1e-6):
    p = np.clip(p, eps, 1 - eps)
    return logit(p)

def rank01(a):
    s = pd.Series(a).rank(method="average")
    return (s - s.min()) / (s.max() - s.min() + 1e-12)

Z_tr = pd.DataFrame(P_tr, columns=use_cols, index=oof_meta.index)
Z_te = pd.DataFrame(P_te, columns=use_cols, index=test_meta.index)
for c in use_cols:
    Z_tr[f"{c}_logit"] = safe_logit(Z_tr[c].values)
    Z_tr[f"{c}_rank"]  = rank01(Z_tr[c].values).values
    Z_te[f"{c}_logit"] = safe_logit(Z_te[c].values)
    Z_te[f"{c}_rank"]  = rank01(Z_te[c].values).values

# me quedo con proba + logit (deja rank si querés; a veces suma, a veces no)
keep_cols = [c for c in Z_tr.columns if (c in use_cols) or c.endswith("_logit")]
sc = StandardScaler()
Z_tr_s = Z_tr.copy()
Z_te_s = Z_te.copy()
Z_tr_s[keep_cols] = sc.fit_transform(Z_tr[keep_cols])
Z_te_s[keep_cols] = sc.transform(Z_te[keep_cols])

meta_en = LogisticRegressionCV(
    Cs=20, cv=5, penalty="elasticnet", solver="saga",
    l1_ratios=[0.2, 0.5, 0.8],
    scoring="average_precision",
    max_iter=5000, n_jobs=4, refit=True
)
meta_en.fit(Z_tr_s[keep_cols], y_tr_arr)
proba_meta = meta_en.predict_proba(Z_te_s[keep_cols])[:,1]
m_meta = metrics_from_proba(y_te_arr, proba_meta)

print("\n=== META Elastic-Net (proba+logit) ===")
print("l1_ratio_:", meta_en.l1_ratio_[0] if hasattr(meta_en, "l1_ratio_") else "n/a")
print("metrics:", m_meta)

# =============== (3) HÍBRIDO: combinación Meta + Blend
best_combo = None
for alpha in [0.3, 0.5, 0.7]:
    proba_h = alpha * proba_meta + (1 - alpha) * proba_blend
    m_h = metrics_from_proba(y_te_arr, proba_h)
    print(f"\n=== HYBRID (alpha={alpha:.1f}) ===")
    print("metrics:", m_h)
    if (best_combo is None) or (m_h["pr_auc"] > best_combo[2]["pr_auc"]):
        best_combo = (alpha, proba_h, m_h)

alpha_star, proba_star, m_star = best_combo
print("\n>>> ELEGIDO POR PR-AUC:", 
      f"Hybrid(alpha={alpha_star})" if m_star["pr_auc"]>=max(m_blend["pr_auc"], m_meta["pr_auc"]) else 
      ("Blend" if m_blend["pr_auc"]>=m_meta["pr_auc"] else "Meta-EN"))
print("PR-AUC:", m_star["pr_auc"])

from sklearn.metrics import average_precision_score

alphas = np.linspace(0.0, 1.0, 41)  # paso de 0.025
best = (-1, None)
best_proba = None
for a in alphas:
    proba = a*proba_meta + (1-a)*proba_blend
    ap = average_precision_score(y_te, proba)
    m_h = metrics_from_proba(y_te_arr, proba)
    if ap > best[0]: 
        best = (ap, a)
        best_proba = m_h
print("Best alpha:", best[1], " | PR-AUC:", best[0])
print(best_proba)

=== BLEND ===
  peso    xgb: 0.308
  peso   lgbm: 0.127
  peso    cat: 0.259
  peso     rf: 0.199
  peso    log: 0.107
metrics: {'pr_auc': 0.8673988513446531, 'roc_auc': 0.8825910471099475, 'f1': 0.7621722846441947, 'accuracy': 0.8101644245142003, 'brier': 0.13709902164473362, 'confusion_matrix': [[677, 79], [175, 407]]}

=== META Elastic-Net (proba+logit) ===
l1_ratio_: 0.8
metrics: {'pr_auc': 0.8720532213407559, 'roc_auc': 0.8853979163257513, 'f1': 0.7621722846441947, 'accuracy': 0.8101644245142003, 'brier': 0.13366858982816102, 'confusion_matrix': [[677, 79], [175, 407]]}

=== HYBRID (alpha=0.3) ===
metrics: {'pr_auc': 0.8699600357949352, 'roc_auc': 0.8842933507881963, 'f1': 0.7663551401869159, 'accuracy': 0.8131539611360239, 'brier': 0.1356542326079534, 'confusion_matrix': [[678, 78], [172, 410]]}

=== HYBRID (alpha=0.5) ===
metrics: {'pr_auc': 0.8709407561741134, 'roc_auc': 0.885018363970254, 'f1': 0.7679403541472507, 'accuracy': 0.8139013452914798, 'brier': 0.13488897300810393, '

In [23]:
# ===========================================
# Elegir α óptimo, guardar artefactos y leaderboard (blend/meta/hybrid)
# ===========================================
import os, json, time, joblib
import numpy as np
import pandas as pd
from sklearn.metrics import (
    average_precision_score, roc_auc_score, f1_score, accuracy_score, brier_score_loss, confusion_matrix
)

def metrics_from_proba(y_true, y_proba, thr=0.5):
    y_pred = (y_proba >= thr).astype(int)
    return dict(
        pr_auc=float(average_precision_score(y_true, y_proba)),
        roc_auc=float(roc_auc_score(y_true, y_proba)),
        f1=float(f1_score(y_true, y_pred)),
        accuracy=float(accuracy_score(y_true, y_pred)),
        brier=float(brier_score_loss(y_true, y_proba)),
        confusion_matrix=confusion_matrix(y_true, y_pred).tolist(),
    )

# ---------- 1) α óptimo para el híbrido (entre proba_meta y proba_blend)
alphas = np.linspace(0.0, 1.0, 41)  # paso 0.025
best_alpha, best_ap = None, -1.0
for a in alphas:
    proba_h = a * proba_meta + (1 - a) * proba_blend
    ap = average_precision_score(y_te, proba_h)
    if ap > best_ap:
        best_ap = ap
        best_alpha = a

proba_hybrid = best_alpha * proba_meta + (1 - best_alpha) * proba_blend

# ---------- 2) Métricas
m_blend  = dict(model="stack_blend",  **metrics_from_proba(y_te, proba_blend))
m_meta   = dict(model="stack_meta_en", **metrics_from_proba(y_te, proba_meta))
m_hybrid = dict(model=f"stack_hybrid_alpha_{best_alpha:.3f}", **metrics_from_proba(y_te, proba_hybrid))

print("[BLEND]  ", m_blend)
print("[META EN]", m_meta)
print(f"[HYBRID α={best_alpha:.3f}]", m_hybrid)

# ---------- 3) Guardado a disco
STAMP   = time.strftime("%Y%m%d_%H%M%S")
EXP_DIR = f"experiments/stacking_plus_v1"
os.makedirs(EXP_DIR, exist_ok=True)

# preds
pd.DataFrame({
    "y_true": y_te.values,
    #"proba_blend":  proba_blend,
    #"proba_meta":   proba_meta,
    "proba_hybrid": proba_hybrid
}).to_csv(os.path.join(EXP_DIR, "test_predictions.csv"), index=False)

# metrics.json (merge)
metrics_path = os.path.join(EXP_DIR, "metrics.json")
metrics_all = {
    #"stack_blend": m_blend,
    #"stack_meta_en": m_meta,
    f"stack_hybrid_alpha_{best_alpha:.3f}": m_hybrid
}
with open(metrics_path, "w") as f:
    json.dump(metrics_all, f, indent=2)

# pesos del blend y α
with open(os.path.join(EXP_DIR, "blend_weights.json"), "w") as f:
    json.dump({name: float(w) for name, w in zip(use_cols, w_star)}, f, indent=2)
with open(os.path.join(EXP_DIR, "hybrid_alpha.txt"), "w") as f:
    f.write(str(best_alpha))

# meta Elastic-Net
joblib.dump(meta_en, os.path.join(EXP_DIR, "meta_elasticnet.pkl"))

# ---------- 4) Leaderboard (append 3 filas)
os.makedirs("experiments", exist_ok=True)
lb_path = "experiments/leaderboard.csv"
rows = []
for m in (m_blend, m_meta, m_hybrid):
    rows.append({
        "timestamp": time.strftime("%Y-%m-%d %H:%M:%S"),
        "model": m["model"],
        "accuracy": m["accuracy"],
        "roc_auc": m["roc_auc"],
        "pr_auc": m["pr_auc"],
        "f1": m["f1"],
        "brier": m["brier"],
        "exp_dir": EXP_DIR
    })
df_lb = pd.DataFrame(rows)
if os.path.exists(lb_path):
    df_lb.to_csv(lb_path, mode="a", header=False, index=False)
else:
    df_lb.to_csv(lb_path, index=False)

print("\nArtefactos guardados en:", EXP_DIR)


[BLEND]   {'model': 'stack_blend', 'pr_auc': 0.8673988513446531, 'roc_auc': 0.8825910471099475, 'f1': 0.7621722846441947, 'accuracy': 0.8101644245142003, 'brier': 0.13709902164473362, 'confusion_matrix': [[677, 79], [175, 407]]}
[META EN] {'model': 'stack_meta_en', 'pr_auc': 0.8720532213407559, 'roc_auc': 0.8853979163257513, 'f1': 0.7621722846441947, 'accuracy': 0.8101644245142003, 'brier': 0.13366858982816102, 'confusion_matrix': [[677, 79], [175, 407]]}
[HYBRID α=0.950] {'model': 'stack_hybrid_alpha_0.950', 'pr_auc': 0.8720648161614923, 'roc_auc': 0.8854706449208166, 'f1': 0.7628865979381443, 'accuracy': 0.8109118086696562, 'brier': 0.13374609320060443, 'confusion_matrix': [[678, 78], [175, 407]]}

Artefactos guardados en: experiments/stacking_plus_v1


In [25]:
digraph StackingHibrido {
  graph [rankdir=LR, fontsize=12, label="Ensamble Híbrido: Stacking + Blending", labelloc=top];
  node  [shape=box, style=rounded, fontsize=11];
  edge  [fontsize=10];

  subgraph cluster_data {
    label="Datos";
    color="#aaaaaa";
    X_train [label="X (train)"];
    y_train [label="y (train)"];
    X_test  [label="X (test)"];
  }

  subgraph cluster_base {
    label="Modelos base (k-fold OOF)";
    color="#6ca0dc";
    style="rounded";
    XGB  [label="XGBoost"];
    LGBM [label="LightGBM"];
    CAT  [label="CatBoost"];
    RF   [label="Random Forest"];
    LOG  [label="Logistic Regression"];
  }

  subgraph cluster_oof {
    label="Predicciones OOF (train) + Predicciones (test)";
    color="#8fd694";
    style="rounded";
    OOF_XGB [label="p_xgb^OOF (train)\np_xgb^test (test)"];
    OOF_LGBM[label="p_lgbm^OOF (train)\np_lgbm^test (test)"];
    OOF_CAT [label="p_cat^OOF (train)\np_cat^test (test)"];
    OOF_RF  [label="p_rf^OOF (train)\np_rf^test (test)"];
    OOF_LOG [label="p_log^OOF (train)\np_log^test (test)"];
  }

  subgraph cluster_meta {
    label="Meta-features";
    color="#f7c97f";
    style="rounded";
    META_TRAIN [label="Z_train = [p^OOF, logit(p^OOF), rank(p^OOF)]"];
    META_TEST  [label="Z_test  = [p^test, logit(p^test), rank(p^test)]"];
  }

  subgraph cluster_metalearner {
    label="Meta-aprendiz";
    color="#ff9e9e";
    style="rounded";
    META_EN   [label="Logistic Regression\n(Elastic-Net)"];
    PMETA     [label="p_meta (test)"];
  }

  subgraph cluster_blend {
    label="Blending (max PR-AUC en holdout meta)";
    color="#c39bd3";
    style="rounded";
    WEIGHTS   [label="Optimización de pesos\nw >= 0, sum w = 1"];
    PBLEND    [label="p_blend (test) = Σ w_i · p_i^test"];
  }

  subgraph cluster_hybrid {
    label="Híbrido final";
    color="#a3d2ca";
    style="rounded";
    HYB       [label="p_hybrid = α·p_meta + (1−α)·p_blend"];
  }

  # Flujo
  X_train -> XGB; X_train -> LGBM; X_train -> CAT; X_train -> RF; X_train -> LOG;
  y_train -> XGB; y_train -> LGBM; y_train -> CAT; y_train -> RF; y_train -> LOG;

  XGB  -> OOF_XGB;  LGBM -> OOF_LGBM; CAT -> OOF_CAT; RF -> OOF_RF; LOG -> OOF_LOG;
  X_test -> OOF_XGB; X_test -> OOF_LGBM; X_test -> OOF_CAT; X_test -> OOF_RF; X_test -> OOF_LOG;

  OOF_XGB  -> META_TRAIN;  OOF_LGBM -> META_TRAIN;  OOF_CAT -> META_TRAIN;  OOF_RF -> META_TRAIN;  OOF_LOG -> META_TRAIN;
  OOF_XGB  -> META_TEST;   OOF_LGBM -> META_TEST;   OOF_CAT -> META_TEST;   OOF_RF -> META_TEST;   OOF_LOG -> META_TEST;

  META_TRAIN -> META_EN;
  META_EN -> PMETA;
  META_TEST  -> META_EN;

  # Blending usa directamente las probas de las bases
  OOF_XGB -> WEIGHTS; OOF_LGBM -> WEIGHTS; OOF_CAT -> WEIGHTS; OOF_RF -> WEIGHTS; OOF_LOG -> WEIGHTS;
  WEIGHTS  -> PBLEND;

  # Híbrido
  PMETA -> HYB;
  PBLEND -> HYB;

  # Leyenda
  subgraph cluster_legend {
    label="Leyenda";
    color="#dddddd";
    L1 [shape=plaintext, label="OOF: Predicción de un fold por un modelo entrenado sin ese fold (evita leakage).\nMeta-features: concatenación de probas OOF/test de cada base y sus transformaciones (logit, rank).\nMeta-aprendiz: combina señales y mejora calibración.\nBlending: elige pesos para maximizar PR-AUC.\nHíbrido: combina meta y blend con α elegido por PR-AUC."];
  }
}


SyntaxError: invalid syntax (2745242668.py, line 1)